# TP12 - Clasificación de imágenes usando CNN (versión 2021)


* Información del sistema

In [4]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic


In [5]:
!nvidia-smi

Sun Oct 24 13:33:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------



*   Montaje del Drive





In [2]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


 * Importamos las librerías que vamos a utilizar

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

 * Vamos al directorio de nuestro dataset

In [7]:
!ls /content/drive/MyDrive/colab/dataset/

final  test  train


 * Creamos la clase que usaremos para crear los datasets.
 * Creamos el conjunto de entrenamiento y el de test.

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/colab/dataset/train',
target_size = (64, 64),
batch_size = 32,
class_mode = 'binary')

test_set = train_datagen.flow_from_directory('/content/drive/MyDrive/colab/dataset/test',
target_size = (64, 64),
batch_size = 32,
class_mode = 'binary')

final_set = test_datagen.flow_from_directory('/content/drive/MyDrive/colab/dataset/final',
target_size = (64, 64),
batch_size = 1,
class_mode = None)



Found 8544 images belonging to 2 classes.
Found 8116 images belonging to 2 classes.
Found 10 images belonging to 1 classes.


 * Creamos nuestro modelo

In [10]:
model = tf.keras.models.Sequential([            
  tf.keras.layers.experimental.preprocessing.Resizing(64, 64, interpolation='bilinear'),                          
  tf.keras.layers.Conv2D(6, (6, 6), activation='relu',                          
                         input_shape=(64, 64, 3)),                              
  tf.keras.layers.Conv2D(12, (5, 5), strides=(2, 2), activation='relu'),        
  tf.keras.layers.Conv2D(24, (4, 4), strides=(2, 2), activation='relu'),        
  tf.keras.layers.Flatten(),                                                    
  tf.keras.layers.Dropout(rate=.25),                                            
  tf.keras.layers.Dense(200, activation='relu'),                                
  tf.keras.layers.Dense(2, activation='softmax')                               
])                                                                              
                                                                                
optimizer = tf.keras.optimizers.Adam(decay=.0001)

model.compile(optimizer=optimizer,                                              
              loss='sparse_categorical_crossentropy',                           
              metrics=['accuracy'])

 * Entrenamos el modelo en el conjunto de entrenamiento

In [11]:
history = model.fit(training_set, epochs = 7)

Epoch 1/7
267/267 [==============================] - 1691s 6s/step - loss: 0.1117 - accuracy: 0.9553
Epoch 2/7
267/267 [==============================] - 123s 457ms/step - loss: 0.0732 - accuracy: 0.9765
Epoch 3/7
267/267 [==============================] - 125s 467ms/step - loss: 0.0551 - accuracy: 0.9809
Epoch 4/7
267/267 [==============================] - 121s 454ms/step - loss: 0.0471 - accuracy: 0.9833
Epoch 5/7
267/267 [==============================] - 122s 459ms/step - loss: 0.0403 - accuracy: 0.9860
Epoch 6/7
267/267 [==============================] - 121s 451ms/step - loss: 0.0348 - accuracy: 0.9870
Epoch 7/7
267/267 [==============================] - 120s 451ms/step - loss: 0.0323 - accuracy: 0.9889


* Evaluamos sobre algunas imágenes de test individuales.

In [17]:
score = model.evaluate(test_set)

254/254 [==============================] - 92s 361ms/step - loss: 0.0183 - accuracy: 0.9942


* Vemos el resultado del modelo


In [18]:
model.summary()
print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 64, 64, None)      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 59, 59, 6)         654       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 12)        1812      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 24)        4632      
_________________________________________________________________
flatten (Flatten)            (None, 4056)              0         
_________________________________________________________________
dropout (Dropout)            (None, 4056)              0         
_________________________________________________________________
dense (Dense)                (None, 200)               8

 * Evaluamos dos imágenes del conjunto "final", que no contiene imágenes del conjunto "train" ni "test".

In [30]:
for i in range(2):
  final = model.predict(final_set[i])[0]
  print(final)
  i = i+1

[9.9987137e-01 1.2865248e-04]
[0.9845791  0.01542084]


 * Guardamos nuestro modelo

In [14]:
model.save('/content/drive/MyDrive/models/model_TP12.h5')